In [1]:
import os
from fancy import Data, Model, Analysis

In [2]:
'''Setting up'''
# Define location of Stan files
stan_path = '../../stan/'

# Define file containing source catalogue information
source_file = '../../data/sourcedata.h5'
uhecr_file = '../../data/UHECRdata.h5'

# make output directory if it doesnt exist
if not os.path.isdir("output"):
    os.mkdir("output")

# source_types = ["SBG_23", "2FHL_250Mpc", "swift_BAT_213"]
source_types = ["SBG_23"]

# detector_types = ["auger2010", "auger2014", "TA2015"]
# detector_type = "auger2014"
detector_type = "TA2015"

# set random seed
random_seed = 19990308

# flag to control showing plots or not
show_plot = True

In [3]:
'''set detector and detector properties'''
if detector_type == "TA2015":
    from fancy.detector.TA2015 import detector_properties, alpha_T, M, Eth
elif detector_type == "auger2014":
    from fancy.detector.auger2014 import detector_properties, alpha_T, M, Eth
elif detector_type == "auger2010":
    from fancy.detector.auger2010 import detector_properties, alpha_T, M, Eth
else:
    raise Exception("Undefined detector type!")

In [4]:
'''Fit arrival model'''
for source_type in source_types:
    # table file
    table_file = '../../tables/tables_{0}_{1}.h5'.format(source_type, detector_type)    
    # define output files
    arrival_output_file = 'output/arrival_fit_{0}_{1}.h5'.format(source_type, detector_type)

    # construct Dataset
    data = Data()
    data.add_source(source_file, source_type)
    data.add_uhecr(uhecr_file, detector_type)
    data.add_detector(detector_properties)
    # data.show();

    # construct arrival model obejct
    arrival_model = stan_path + 'arrival_direction_model.stan'
    model = Model(model_filename = arrival_model, include_paths = stan_path)
    model.compile()
    model.input(Eth = Eth) # EeV

    # What is happening 
    # summary = b'Fit of the joint model to the Auger data' 
    summary = b'Fit of the arrival direction model to data' 
        
    # Define an Analysis object to bring together Data and Model objects
    analysis = Analysis(data, model, analysis_type = 'joint', 
                        filename = arrival_output_file, summary = summary)

    # Define location of pre-computed values used in fits 
    # (see relevant notebook for how to make these files) 
    # Each catalogue has a file of pre-computed values
    analysis.use_tables(table_file)

    # Fit the Stan model
    fit = analysis.fit_model(chains = 16, iterations = 500, seed = random_seed)

    # Save to analysis file
    analysis.save()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL model_ef5d94b32aaef7f1c9cb11392672aecd NOW.
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
In file included from /opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /tmp/pystan_ef04upjj/stanfit4model_ef5d94b32aaef7f1c9cb11392672aecd_8032604275740494767.cpp:706:
/opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~
In file i

Performing fitting...

Gradient evaluation took 0.001295 seconds
1000 transitions using 10 leapfrog steps per transition would take 12.95 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001288 seconds
1000 transitions using 10 leapfrog steps per transition would take 12.88 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001288 seconds
1000 transitions using 10 leapfrog steps per transition would take 12.88 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.00134 seconds
1000 transitions using 10 leapfrog steps per transition would take 13.4 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001468 seconds
1000 transitions using 10 leapfrog steps per transition would take 14.68 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.001364 seconds
1000 transitions using 10 leapfrog steps per transition would take 13.64 seconds.
Adjust your expectations accor

Iteration: 500 / 500 [100%]  (Sampling)

 Elapsed Time: 2.75158 seconds (Warm-up)
               2.75049 seconds (Sampling)
               5.50207 seconds (Total)

Checking all diagnostics...
Done!


In [5]:
'''Fit joint model'''
for source_type in source_types:
    # table file
    table_file = '../../tables/tables_{0}_{1}.h5'.format(source_type, detector_type)    
    # define output files
    joint_output_file = 'output/joint_fit_{0}_{1}.h5'.format(source_type, detector_type)

    # construct Dataset
    data = Data()
    data.add_source(source_file, source_type)
    data.add_uhecr(uhecr_file, detector_type)
    data.add_detector(detector_properties)

    # if show_plot:
    #     data.show()

    joint_model = stan_path + 'joint_model.stan'
    model = Model(model_filename = joint_model, include_paths = stan_path)
    model.compile()
    model.input(Eth = Eth) # EeV

    # What is happening 
    # summary = b'Fit of the joint model to the Auger data' 
    summary = b'Fit of the joint model to data' 
        
    # Define an Analysis object to bring together Data and Model objects
    analysis = Analysis(data, model, analysis_type = 'joint', 
                        filename = joint_output_file, summary = summary)

    # Define location of pre-computed values used in fits 
    # (see relevant notebook for how to make these files) 
    # Each catalogue has a file of pre-computed values
    analysis.use_tables(table_file)

    # Fit the Stan model
    fit = analysis.fit_model(chains = 16, iterations = 500, seed = random_seed)

    # Save to analysis file
    analysis.save()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL model_471ee52d0bfdf977d03b7a48b986319e NOW.
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
In file included from /opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /tmp/pystan_vossej66/stanfit4model_471ee52d0bfdf977d03b7a48b986319e_4557306447566338502.cpp:706:
/opt/miniconda3/envs/uhecr_env/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~
In file i

Performing fitting...

Gradient evaluation took 0.002103 seconds
1000 transitions using 10 leapfrog steps per transition would take 21.03 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.002631 seconds
1000 transitions using 10 leapfrog steps per transition would take 26.31 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.003185 seconds
1000 transitions using 10 leapfrog steps per transition would take 31.85 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.002721 seconds
1000 transitions using 10 leapfrog steps per transition would take 27.21 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.003665 seconds
1000 transitions using 10 leapfrog steps per transition would take 36.65 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.002224 seconds
1000 transitions using 10 leapfrog steps per transition would take 22.24 seconds.
Adjust your expectations acc

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Iteration: 500 / 500 [100%]  (Sampling)

 Elapsed Time: 17.1341 seconds (Warm-up)
               10.0647 seconds (Sampling)
               27.1988 seconds (Total)

Checking all diagnostics...
Done!
